# E-LEARNING PLATFORM DATABASE


## Creating Database 


In [11]:
!pip install Faker

In [ ]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import pandas as pd
from faker import Faker 
fake = Faker()
import random
import time

In [5]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']
print("Connected to MongoDB successfully!")
print(f"Database: {db.name}")
print(f"Collections: {db.list_collection_names()}")

Connected to MongoDB successfully!
Database: eduhub_db
Collections: []


## Creating Collections With Validations

In [15]:
# Droping existing collections for clean start if exists
for collection in ['users', 'courses', 'enrollments', 'lessons', 'assignments', 'submissions']:
    db[collection].drop()

#Users Collection with Validation
users_validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['userId', 'email', 'firstName', 'lastName', 'role', 'dateJoined', 'isActive'],
        'properties': {
            'userId': {'bsonType': 'string'},
            'email': {
                'bsonType': 'string',
                'pattern': '^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$'
            },
            'firstName': {'bsonType': 'string', 'minLength': 1},
            'lastName': {'bsonType': 'string', 'minLength': 1},
            'role': {'enum': ['student', 'instructor']},
            'dateJoined': {'bsonType': 'date'},
            'isActive': {'bsonType': 'bool'},
            'profile': {
                'bsonType': 'object',
                'properties': {
                    'bio': {'bsonType': 'string'},
                    'avatar': {'bsonType': 'string'},
                    'skills': {
                        'bsonType': 'array',
                        'items': {'bsonType': 'string'}
                    }
                }
            }
        }
    }
}

db.create_collection('users', validator=users_validator)
print("Users collection created")

# Courses Collection with Validation
courses_validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': [
            'courseId', 'title', 'instructorId',
            'level', 'isPublished', 'createdAt'
        ],
        'properties': {
            '_id': {'bsonType': 'objectId'},
            'courseId': {'bsonType': 'string'},
            'title': {'bsonType': 'string', 'minLength': 1},
            'description': {'bsonType': 'string'},
            'instructorId': {'bsonType': 'string'},  # reference to users
            'category': {'bsonType': 'string'},
            'level': {'enum': ['beginner', 'intermediate', 'advanced']},
            'duration': {'bsonType': 'number', 'minimum': 0},  # in hours
            'price': {'bsonType': 'number', 'minimum': 0},
            'tags': {
                'bsonType': 'array',
                'items': {'bsonType': 'string'}
            },
            'createdAt': {'bsonType': 'date'},
            'updatedAt': {'bsonType': 'date'},
            'isPublished': {'bsonType': 'bool'}
        }
    }
}

db.create_collection('courses', validator=courses_validator)
print("Courses collection created")


#Enrollment Collection with Validation
enrollments_validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['enrollmentId', 'studentId', 'courseId', 'enrolledAt', 'status'],
        'properties': {
            '_id': {'bsonType': 'objectId'},
            'enrollmentId': {'bsonType': 'string'},
            'studentId': {'bsonType': 'string'},
            'courseId': {'bsonType': 'string'},
            'enrolledAt': {'bsonType': 'date'},
            'status': {'enum': ['active', 'completed', 'dropped']},
            'progress': {'bsonType': 'number', 'minimum': 0, 'maximum': 100},
            'completedAt': {'bsonType': ['date', 'null']},
            'lastAccessedAt': {'bsonType': 'date'}
        }
    }
}

db.create_collection('enrollments', validator=enrollments_validator)
print("Enrollments collection created")

#Lesson Collection with Validation
lessons_validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['lessonId', 'courseId', 'title', 'content', 'order'],
        'properties': {
            '_id': {'bsonType': 'objectId'},
            'lessonId': {'bsonType': 'string'},
            'courseId': {'bsonType': 'string'},  # reference to courses
            'title': {'bsonType': 'string', 'minLength': 1},
            'content': {'bsonType': 'string'},
            'order': {'bsonType': 'number', 'minimum': 1}, # lesson sequence
            'resources': {
                'bsonType': 'array',
                'items': {'bsonType': 'string'}  # URLs or file links
            },
            'createdAt': {'bsonType': 'date'},
            'updatedAt': {'bsonType': 'date'}
        }
    }
}

db.create_collection('lessons', validator=lessons_validator)
print("Lessons collection created")

#Assignment Collection with Validation
assignments_validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['assignmentId', 'courseId', 'title', 'description', 'dueDate'],
        'properties': {
            '_id': {'bsonType': 'objectId'},
            'assignmentId': {'bsonType': 'string'},
            'courseId': {'bsonType': 'string'},  # reference to course
            'title': {'bsonType': 'string', 'minLength': 1},
            'description': {'bsonType': 'string'},
            'dueDate': {'bsonType': 'date'},
            'createdAt': {'bsonType': 'date'},
            'updatedAt': {'bsonType': 'date'},
            'maxScore': {'bsonType': 'number', 'minimum': 0}
        }
    }
}

db.create_collection('assignments', validator=assignments_validator)
print("Assignments collection created")

#Submission Collection with Validation
submissions_validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['submissionId', 'assignmentId', 'studentId', 'submittedAt'],
        'properties': {
            '_id': {'bsonType': 'objectId'},
            'submissionId': {'bsonType': 'string'},
            'assignmentId': {'bsonType': 'string'},
            'studentId': {'bsonType': 'string'},
            'submittedAt': {'bsonType': 'date'},
            'content': {'bsonType': 'string'},
            'fileUrl': {'bsonType': 'string'},
            'grade': {
                'bsonType': ['number', 'null'],
                'minimum': 0
            },
            'feedback': {'bsonType': 'string'},
            'gradedAt': {'bsonType': ['date', 'null']},
            'gradedBy': {'bsonType': 'string'},
            'status': {
                'enum': ['submitted', 'graded', 'returned']
            }
        }
    }
}

db.create_collection('submissions', validator=submissions_validator)
print("Submissions collection created")


print("\nAll collections created successfully!")
print(f"Collections: {db.list_collection_names()}")


Users collection created
Courses collection created
Enrollments collection created
Lessons collection created
Assignments collection created
Submissions collection created

All collections created successfully!
Collections: ['lessons', 'submissions', 'assignments', 'courses', 'users', 'enrollments']


## Unique Indexes For Collection

In [18]:
# Enforce uniqueness on userId and email in users collection
collections_indexes = {
    "users": ["userId", "email"],
    "courses": ["courseId"],
    "enrollments": ["enrollmentId"],
    "lessons": ["lessonId"],
    "assignments": ["assignmentId"],
    "submissions": ["submissionId"]
}

for collection_name, fields in collections_indexes.items():
    if collection_name in db.list_collection_names():
        collection = db[collection_name]
        for field in fields:
            try:
                collection.create_index(field, unique=True)
                print(f"Unique index created on {collection_name}.{field}")
            except Exception as e:
                print(f"Could not create index on {collection_name}.{field}: {e}")
    else:
        print(f"Collection {collection_name} does not exist")

Unique index created on users.userId
Unique index created on users.email
Unique index created on courses.courseId
Unique index created on enrollments.enrollmentId
Unique index created on lessons.lessonId
Unique index created on assignments.assignmentId
Unique index created on submissions.submissionId
